In [1]:
!pip install descartes
!pip install geopandas
!pip install contextily

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 64.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.4 MB/s eta 0:00:00


In [2]:
from osgeo import gdal
from osgeo import ogr# 矢量处理数据包(shp,json等)
from osgeo import osr# 投影坐标包
from osgeo import gdal_array
from osgeo import gdalconst
import geopandas as gpd
import numpy as np

import json# json是存储和交换文本信息的语法，类似 XML
import requests# 网络数据获取(开源数据，不开源就用爬虫)
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
newpath = "output_sobel.tif"
path = '/content/drive/MyDrive/AI data/GIS/Aerial image/austin30.tif'
path1 ='/content/drive/MyDrive/AI data/GIS/Aerial image/austin30_sub.tif'

In [4]:
def readTif(path):
  dataset = gdal.Open(path)
  im_width = dataset.RasterXSize
  im_height = dataset.RasterYSize
  im_proj = dataset.GetProjection()
  im_Geotrans = dataset.GetGeoTransform()
  im_data = dataset.ReadAsArray(0,0,im_width,im_height)

  del dataset
  return im_proj,im_Geotrans,im_data

def writeTif(newpath,im_data,im_Geotrans,im_proj,datatype):
  # 多波段
  if len(im_data.shape) == 3:
    im_bands,im_height,im_width = im_data.shape
  # 单波段
  else:
    im_bands,(im_height,im_width) = 1,im_data.shape
  # 定义driver
  driver = gdal.GetDriverByName('GTiff')
  # 生成Raster
  new_dataset = driver.Create(newpath,im_width,im_height,im_bands,datatype)
  # set Geotransform&Projection
  new_dataset.SetGeoTransform(im_Geotrans)
  new_dataset.SetProjection(im_proj)

  # 写入数据
  if im_bands == 1:
    new_dataset.GetRasterBand(1).WriteArray(im_data)
  else:
    for i in range(im_bands):
      new_dataset.GetRasterBand(i+1).WriteArray(im_data[i])
  
  del new_dataset



In [5]:

#均值滤波
def convolution(im_data,kernel):

  gap = kernel.shape[0]//2

  # 多波段
  if len(im_data.shape) == 3:
    im_bands,im_height,im_width = im_data.shape
  # 单波段
  else:
    im_bands,(im_height,im_width) = 1,im_data.shape

  im_data = im_data.astype(np.float)
  temp = im_data.copy()

  if im_bands == 1:
    for i in range(gap,im_height-gap):
      for j in range(gap,im_width-gap):
        cell = 0
        for m in range(-gap,gap+1):
          for n in range(-gap,gap+1):
            cell+=temp[i+m][j+n]*kernel[m+1][n+1]
        im_data[i][j] = cell
  else:
    for k in range(im_bands):
      for i in range(gap,im_height-gap):
        for j in range(gap,im_width-gap):
          cell = 0
          for m in range(-gap,gap+1):
            for n in range(-gap,gap+1):
              cell+=temp[k][i+m][j+n]*kernel[m+1][n+1]
          im_data[k][i][j] = cell
  return im_data

In [7]:
# 边缘检测算子

def sobel(im_data):
  gx = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
  gy = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
  gx_data = convolution(im_data,gx)
  gy_data = convolution(im_data,gy)

  return np.abs(gx_data) + np.abs(gy_data)

In [8]:
kernel_mean  = np.array([[1/9,1/9,1/9],[1/9,1/9,1/9],[1/9,1/9,1/9]])

im_proj,im_Geotrans,im_data = readTif(path1)
# im_data = convolution(im_data,kernel=kernel_mean)
im_data = sobel(im_data)
writeTif(newpath,im_data,im_Geotrans,im_proj,gdal.GDT_UInt16)

<ipython-input-5-e02ea3f8167a>:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  im_data = im_data.astype(np.float)
